In [14]:
import spotipy
import spotipy.util as util
import pickle
import sys
import requests
from tqdm import tqdm
import os
import pandas as pd
import numpy as np
from multiprocessing import Pool
from fuzzywuzzy import fuzz
import string
import json
import unidecode
import sys
!{sys.executable} -m pip install pytrends
import time
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta
from pytrends.request import TrendReq


In [15]:
#Read song data frame from pkl file, needs to change it in step 1
with open('song_df_100.pkl', 'rb') as f:
        song_df = pickle.load(f)

In [4]:
song_df

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,spotify_acousticness,spotify_instrumentalness,spotify_liveness,spotify_valence,spotify_tempo,spotify_time_signature,spotify_album_id,spotify_album_release_date,spotify_album_release_date_precision,album_label
0,Lewis Capaldi,Someone You Loved,False,1,2019-11-16,7qEHsqek33rTcFNT9PFqLf,False,182160.0,1.0,4.0,...,0.7510,0.00000,0.1050,0.4460,109.891,4.0,5658aM19fA3JVwTK6eQX70,2019-05-17,day,Vertigo Berlin
1,Post Malone,Circles,False,2,2019-11-16,21jGcNKet2qwijlDFuPiPb,False,215280.0,1.0,6.0,...,0.1920,0.00244,0.0863,0.5530,120.042,4.0,4g1ZRSobMefqF6nelkgibi,2019-09-06,day,Republic Records
2,Shawn Mendes & Camila Cabello,Senorita,False,3,2019-11-16,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
3,Lizzo,Truth Hurts,False,4,2019-11-16,5qmq61DAAOUaW8AUo8xKhh,True,173325.0,1.0,13.0,...,0.1100,0.00000,0.1230,0.4120,158.087,4.0,74gSdSHe71q7urGWMMn3qB,2019-05-03,day,Nice Life/Atlantic
4,Selena Gomez,Lose You To Love Me,False,5,2019-11-16,1HfMVBKM75vxSfsQ5VefZ5,False,206458.0,1.0,1.0,...,0.5760,0.00000,0.2100,0.0916,101.993,4.0,3tBkjgxDqAwss76O1YHsSY,2019-10-23,day,Interscope Records
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113178,Spice Girls,Spice Up Your Life,False,79,1998-03-21,5qGwqO0lkbBXw4xNfzT7SF,False,173666.0,1.0,1.0,...,0.0105,0.03410,0.4030,0.7020,124.016,4.0,3sr6lAuO3nmB1u8ZuQgpiX,1997-01-01,day,Virgin Records
113192,Phajja,"So Long (Well, Well, Well)",False,93,1998-03-21,7phS4S15qLjPC7JlFnzWql,False,262333.0,1.0,23.0,...,0.4320,0.00168,0.0446,0.5450,79.913,4.0,6qbvgcaWgMn2BQY6FhdOm2,2018-11-16,day,Warner Music Group - X5 Music Group
113194,Sting & The Police,Roxanne `97 - Puff Daddy Remix,False,95,1998-03-21,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN
113198,Shania Twain,Don't Be Stupid (You Know I Love You),False,99,1998-03-21,None,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN


In [16]:
#Drop lines containing N/A
song_dropna = song_df.dropna().copy()
song_dropna

,artist,title,isnew,rank,date,spotify_id,spotify_explicit,spotify_duration_ms,spotify_disc_number,spotify_track_number,...,spotify_acousticness,spotify_instrumentalness,spotify_liveness,spotify_valence,spotify_tempo,spotify_time_signature,spotify_album_id,spotify_album_release_date,spotify_album_release_date_precision,album_label
0,Lewis Capaldi,Someone You Loved,False,1,2019-11-16,7qEHsqek33rTcFNT9PFqLf,False,182160.0,1.0,4.0,...,0.7510,0.000000,0.1050,0.4460,109.891,4.0,5658aM19fA3JVwTK6eQX70,2019-05-17,day,Vertigo Berlin
1,Post Malone,Circles,False,2,2019-11-16,21jGcNKet2qwijlDFuPiPb,False,215280.0,1.0,6.0,...,0.1920,0.002440,0.0863,0.5530,120.042,4.0,4g1ZRSobMefqF6nelkgibi,2019-09-06,day,Republic Records
3,Lizzo,Truth Hurts,False,4,2019-11-16,5qmq61DAAOUaW8AUo8xKhh,True,173325.0,1.0,13.0,...,0.1100,0.000000,0.1230,0.4120,158.087,4.0,74gSdSHe71q7urGWMMn3qB,2019-05-03,day,Nice Life/Atlantic
4,Selena Gomez,Lose You To Love Me,False,5,2019-11-16,1HfMVBKM75vxSfsQ5VefZ5,False,206458.0,1.0,1.0,...,0.5760,0.000000,0.2100,0.0916,101.993,4.0,3tBkjgxDqAwss76O1YHsSY,2019-10-23,day,Interscope Records
5,Lizzo,Good As Hell,False,6,2019-11-16,3Yh9lZcWyKrK9GjbhuS0hR,False,159735.0,1.0,1.0,...,0.2560,0.000000,0.4420,0.5350,95.982,4.0,4CA8GvVF7swZafg0zYh9qq,2016-03-09,day,Nice Life/Atlantic
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113099,The Rolling Stones,Saint Of Me,False,100,1998-03-28,6OEpSYq8wdqnl01hsoQGQY,False,314626.0,1.0,8.0,...,0.0049,0.044300,0.0895,0.4940,121.541,4.0,6lbQFxYqubkf4GbEtXARE7,1997-09-29,day,Polydor Records
113147,Nu Flavor,Heaven,False,48,1998-03-21,0zLMsK6UeAP4KrbekFbHvC,False,298133.0,1.0,3.0,...,0.8440,0.000176,0.0875,0.3500,96.178,4.0,6YW8UAkoK9RxL08k44HziH,1997,year,Rhino/Warner Records
113178,Spice Girls,Spice Up Your Life,False,79,1998-03-21,5qGwqO0lkbBXw4xNfzT7SF,False,173666.0,1.0,1.0,...,0.0105,0.034100,0.4030,0.7020,124.016,4.0,3sr6lAuO3nmB1u8ZuQgpiX,1997-01-01,day,Virgin Records
113192,Phajja,"So Long (Well, Well, Well)",False,93,1998-03-21,7phS4S15qLjPC7JlFnzWql,False,262333.0,1.0,23.0,...,0.4320,0.001680,0.0446,0.5450,79.913,4.0,6qbvgcaWgMn2BQY6FhdOm2,2018-11-16,day,Warner Music Group - X5 Music Group


In [17]:
from pytrends.request import TrendReq
from random import randint

In [74]:
#this function returns the mean trend of the past n months of a song in song_df 
def google_trend(song_df, song, n):
    trend_date = song_df.iloc[song]["spotify_album_release_date"]
    
    if len(trend_date) > 5:
        dt = datetime.strptime(trend_date, '%Y-%m-%d')
    else:
        dt = datetime.strptime(trend_date, '%Y')
    
    dt_start = dt + relativedelta(months=-n)
   
    trend_artist = song_df.iloc[song]["artist"]
    kw_list = [trend_artist]
    
    #set dataframe, the earliset is 1/1/2004
    earliest_date = '2004-1-1'
    google_earliest = datetime.strptime(earliest_date, '%Y-%m-%d')
    if dt_start >= google_earliest:
        pytrends = TrendReq(hl='en-US', tz=360)
        time_period = dt_start.strftime("%Y") + '-' + dt_start.strftime("%m") + '-' + dt_start.strftime("%d") + ' ' + dt.strftime("%Y") + '-' + dt.strftime("%m") + '-' + dt.strftime("%d")
        pytrends.build_payload(kw_list, cat=0, timeframe= time_period, geo='', gprop='')
        trend = pytrends.interest_over_time()
        if not trend.empty:
            trend_mean=np.mean(trend.iloc[:,0])
            return trend_mean
        else:
            return 0
    elif dt_start < google_earliest and dt > google_earliest:
        pytrends = TrendReq(hl='en-US', tz=360)
        time_period = google_earliest.strftime("%Y") + '-' + google_earliest.strftime("%m") + '-' + google_earliest.strftime("%d") + ' ' + dt.strftime("%Y") + '-' + dt.strftime("%m") + '-' + dt.strftime("%d")
        pytrends.build_payload(kw_list, cat=0, timeframe= time_period, geo='', gprop='')
        trend = pytrends.interest_over_time()
        if not trend.empty:
            trend_mean=np.mean(trend.iloc[:,0])
            return trend_mean
        else:
            return 0
    elif dt <= google_earliest:
        trend_mean = -1
        return trend_mean

#test    
print(google_trend(song_df, 0, 3))

34.544444444444444


In [152]:
# fill in trend feature from mth row to nth row
def trend_df(google_trend_df, song_dropna, m, n):

    i=0
    for song in range(m, n, 1):
        
        try: 
            google_trend_df = google_trend_df.append({'google_trend': google_trend(song_dropna, song, 3)} , ignore_index=True)
        except requests.exceptions.Timeout:
            print("Timeout occurred")
        time.sleep(randint(3,10))

        print("song_df row No.", end =" ")
        print(song)
        print("Mean trend: ", end =" ")
        print(google_trend_df.iloc[i+1]["google_trend"])
        i=i+1
    
    #save as pkl file
    with open('google_trend_df'+'_'+str(m)+'_'+str(n)+'.pkl', 'wb') as f:
        pickle.dump(google_trend_df, f)

In [160]:
#trend for the last 18 rows
data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 4951, len(song_dropna))

song_df row No. 4951
Mean trend:  -1
song_df row No. 4952
Mean trend:  30.16304347826087
song_df row No. 4953
Mean trend:  -1
song_df row No. 4954
Mean trend:  -1
song_df row No. 4955
Mean trend:  -1
song_df row No. 4956
Mean trend:  50.56989247311828
song_df row No. 4957
Mean trend:  -1
song_df row No. 4958
Mean trend:  -1
song_df row No. 4959
Mean trend:  -1
song_df row No. 4960
Mean trend:  -1
song_df row No. 4961
Mean trend:  -1
song_df row No. 4962
Mean trend:  -1
song_df row No. 4963
Mean trend:  -1
song_df row No. 4964
Mean trend:  -1
song_df row No. 4965
Mean trend:  -1
song_df row No. 4966
Mean trend:  -1
song_df row No. 4967
Mean trend:  8.817204301075268
song_df row No. 4968
Mean trend:  -1


In [161]:
#validate the file is saved correctly
with open('google_trend_df_4951_4969.pkl', 'rb') as f:
        google_trend_df = pickle.load(f)
google_trend_df

,google_trend
0,
1,-1
2,30.163
3,-1
4,-1
5,-1
6,50.5699
7,-1
8,-1
9,-1


trend_feature(song_dropna, 0, 1000)

In [ ]:
##Above is the program
#Belove is how to use the program

In [ ]:
#First 1000 rows (from 0 to 1000), saved as song_dropna_0_1000.pkl 
#I used the old data set which has 4969 rows arfer dropping N/A.

#To get data for the other rows
# Stpe 1 change the picklefile to read at the beginning of this program
# Stpe 2 Change the parameters in the fuction below
# Stpe 3 Change the pkl filename to save
# Stpe 4 Click run

data = {'google_trend':['']} 
google_trend_df = pd.DataFrame(data)
trend_df(google_trend_df, song_dropna, 0, 1000)